In [8]:
import warnings

# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')


In [9]:
import geopandas as gpd
F1 = 'data/상권 영역/서울시 상권분석서비스(영역-상권).shp' 
D1 = gpd.read_file(F1, encoding='utf-8')
D1['geometry']

0       POLYGON ((196955.842 453722.114, 196958.669 45...
1       POLYGON ((201385.380 452513.725, 201385.284 45...
2       POLYGON ((201937.121 452552.083, 201872.576 45...
3       POLYGON ((197550.670 453079.887, 197554.026 45...
4       POLYGON ((197428.009 453548.216, 197436.697 45...
                              ...                        
1645    POLYGON ((211759.866 449221.074, 211779.452 44...
1646    POLYGON ((211924.717 447688.584, 211931.962 44...
1647    POLYGON ((212814.498 448972.661, 212804.818 44...
1648    POLYGON ((212922.264 449886.083, 212922.218 44...
1649    POLYGON ((214945.000 451011.141, 214946.951 45...
Name: geometry, Length: 1650, dtype: geometry

In [10]:
import pandas as pd
df = pd.read_csv('한식음식점.csv')
# df.head()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 84 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상권_구분_코드_명        1378 non-null   object 
 1   상권_코드_명           1378 non-null   object 
 2   총_유동인구_수          1378 non-null   float64
 3   남성_유동인구_수         1378 non-null   float64
 4   여성_유동인구_수         1378 non-null   float64
 5   연령대_10_유동인구_수     1378 non-null   float64
 6   연령대_20_유동인구_수     1378 non-null   float64
 7   시간대_06_11_유동인구_수  1378 non-null   float64
 8   시간대_11_14_유동인구_수  1378 non-null   float64
 9   시간대_14_17_유동인구_수  1378 non-null   float64
 10  시간대_17_21_유동인구_수  1378 non-null   float64
 11  시간대_21_06_유동인구_수  1378 non-null   float64
 12  주중_유동인구_수         1378 non-null   float64
 13  주말_유동인구_수         1378 non-null   float64
 14  연령대_30_40_유동인구_수  1378 non-null   float64
 15  연령대_50_60_유동인구_수  1378 non-null   float64
 16  총_상주인구_수          1378 non-null   int64  


In [12]:
list(df['자치구_코드_명'].unique())

['강북구',
 '강동구',
 '구로구',
 '마포구',
 '성북구',
 '중랑구',
 '강서구',
 '관악구',
 '금천구',
 '용산구',
 '동작구',
 '송파구',
 '도봉구',
 '서대문구',
 '동대문구',
 '은평구',
 '강남구',
 '양천구',
 '광진구',
 '종로구',
 '성동구',
 '노원구',
 '중구',
 '영등포구',
 '서초구']

In [13]:
def create_geodata(df):
    import pandas as pd
    import geopandas as gpd
    import matplotlib.pyplot as plt
    
    # prepare and read data
    F1 = 'data/상권 영역/서울시 상권분석서비스(영역-상권).shp' 
    D1 = gpd.read_file(F1, encoding='utf-8')

    geo_data = pd.merge(df[['자치구_코드_명','상권_코드_명','서비스_업종_코드_명','총_유동인구_수','총_상주인구_수','총_직장_인구_수','유사_업종_점포_수','집객시설수','교통시설수','당월_매출_금액','영역_면적','cluster']],D1,left_on='상권_코드_명',right_on='TRDAR_CD_N')
    geo_data = geo_data[['자치구_코드_명','상권_코드_명','서비스_업종_코드_명','총_유동인구_수','총_상주인구_수','총_직장_인구_수','유사_업종_점포_수','집객시설수','교통시설수','당월_매출_금액','영역_면적','cluster','geometry']]
    geo_data = gpd.GeoDataFrame(geo_data)

    return geo_data

In [14]:
geo_data = create_geodata(df)

In [15]:
import geopandas as gpd

# Assuming 'your_column' is the column you're working with
if isinstance(geo_data['geometry'], gpd.geoseries.GeoSeries):
    geo_data['geometry'] = geo_data['geometry'].buffer(5)
    geo_data['result'] = None
    
    cluster_data = {}  # Dictionary to store DataFrames

    for i in range(geo_data['cluster'].nunique()):
        cluster_data[f'geo_data_{i}'] = geo_data[geo_data['cluster'] == i].reset_index(drop=True)

else:
    print("It's not a GeoSeries.")

In [42]:
def merge_polygon(geo_data,n):
    import geopandas as gpd
    from shapely.ops import unary_union
    data = dict(자치구=[], 상권_코드=[], 서비스_업종_코드_명=[],총_유동인구_수=[], 총_상주인구_수=[], 총_직장_인구_수=[], 유사_업종_점포_수=[], 집객시설수=[], 교통시설수=[], 당월_매출_금액=[],영역_면적=[], cluster=[],geometry=[])
    df = gpd.GeoDataFrame(data)
    # n = code
    cluster = geo_data['cluster'][0]
    service = geo_data['서비스_업종_코드_명'][0]
    # geo_data['result']=None
    while len(geo_data) != 0:
        area_list = []
        while geo_data['result'].sum() != 1:
            geo_data['result'] = None
            for row in range(len(geo_data)):
                if geo_data['geometry'][0].intersects(geo_data['geometry'][row]):
                    geo_data['result'][row]=1
                else :
                    geo_data['result'][row]=0
            geo_data_union = geo_data[geo_data['result']==1]
            mergedPolys = unary_union(geo_data_union['geometry'])
            for value in geo_data_union['자치구_코드_명']:
                area_list.append(value)
            geo_data = geo_data[geo_data['result']==0]
            geo_data.loc[0] = [geo_data_union['자치구_코드_명'][0],n,service,geo_data_union['총_유동인구_수'].sum(),geo_data_union['총_상주인구_수'].sum(),geo_data_union['총_직장_인구_수'].sum(),geo_data_union['유사_업종_점포_수'].sum(),geo_data_union['집객시설수'].sum(),geo_data_union['교통시설수'].sum(),geo_data_union['당월_매출_금액'].sum(),geo_data_union['영역_면적'].sum(),cluster,mergedPolys,1]
        area_list = list(set(area_list))
        for idx in range(len(area_list)):
            df.loc[len(df)] = [area_list[idx],n,service,geo_data['총_유동인구_수'][0],geo_data['총_상주인구_수'][0],geo_data['총_직장_인구_수'][0],geo_data['유사_업종_점포_수'][0],geo_data['집객시설수'][0],geo_data['교통시설수'][0],geo_data['당월_매출_금액'][0],geo_data['영역_면적'][0],cluster,geo_data['geometry'][0]]
        geo_data = geo_data.drop(0, axis=0).reset_index(drop=True)
        n+=1
    return df

In [36]:
merge_polygon(geo_data,1)

,자치구,상권_코드,서비스_업종_코드_명,총_유동인구_수,총_상주인구_수,총_직장_인구_수,유사_업종_점포_수,집객시설수,교통시설수,당월_매출_금액,영역_면적,cluster,geometry
0,강북구,1,한식음식점,1672809.50,4019,316,45,0.0,0.0,3.185594e+09,179020,3,"POLYGON ((201474.814 460537.889, 201474.963 46..."
1,도봉구,1,한식음식점,1672809.50,4019,316,45,0.0,0.0,3.185594e+09,179020,3,"POLYGON ((201474.814 460537.889, 201474.963 46..."
2,강동구,2,한식음식점,8489177.50,26816,3733,152,0.0,0.0,5.643598e+09,596123,3,"POLYGON ((212529.204 448121.917, 212530.873 44..."
3,구로구,3,한식음식점,4125386.00,12509,51423,402,38.0,14.0,3.751723e+10,920022,3,"POLYGON ((190814.981 441984.587, 190812.451 44..."
4,금천구,3,한식음식점,4125386.00,12509,51423,402,38.0,14.0,3.751723e+10,920022,3,"POLYGON ((190814.981 441984.587, 190812.451 44..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,동대문구,603,한식음식점,511157.25,181,73,45,0.0,0.0,3.067470e+09,52646,3,"POLYGON ((203582.205 453451.309, 203582.209 45..."
643,중랑구,604,한식음식점,48659.25,326,3,5,0.0,0.0,9.777474e+07,4443,3,"POLYGON ((206888.891 455544.220, 206898.362 45..."
644,광진구,605,한식음식점,134837.50,302,40,7,0.0,0.0,3.640810e+08,11040,3,"POLYGON ((206222.805 449336.628, 206222.882 44..."
645,성동구,606,한식음식점,662466.25,480,32,80,0.0,0.0,4.830558e+09,23243,3,"POLYGON ((203521.013 450996.104, 203520.750 45..."


In [47]:
merge_polygon_data = {}  # Dictionary to store DataFrames
num = 1
for i in range(len(cluster_data)):
    merge_polygon_data[f'df_{i}'] = merge_polygon(cluster_data[f'geo_data_{i}'],num)
    num +=  merge_polygon_data[f'df_{i}']['상권_코드'].max() - merge_polygon_data[f'df_{i}']['상권_코드'].min() + 1

In [48]:
def concat_geodata(merge_polygon_data):
    for i in range(1,len(merge_polygon_data)):
        if i == 1:
            df = pd.concat([merge_polygon_data['df_0'],merge_polygon_data['df_1']],axis=0)
        else:
            df = pd.concat([df,merge_polygon_data[f'df_{i}']],axis=0)
    # list = []
    # for i in range(1,len(df)+1):
    #     list.append(i)
    # df['상권_코드'] = list
    df = df.reset_index(drop=True)
        
    return df
    
# df_korean_food = pd.concat([merge_polygon_data['df_0'],merge_polygon_data['df_1'],merge_polygon_data['df_2']],axis=0)
# df_korean_food = pd.concat([df_korean_food,merge_polygon_data['df_2']],axis=0)
# df_korean_food = pd.concat([df_korean_food,df_3],axis=0)
# df_korean_food = df_korean_food.reset_index(drop=True)
# len(df_korean_food)
# df_korean_food

In [49]:
df_all = concat_geodata(merge_polygon_data)

In [50]:
df_all

,자치구,상권_코드,서비스_업종_코드_명,총_유동인구_수,총_상주인구_수,총_직장_인구_수,유사_업종_점포_수,집객시설수,교통시설수,당월_매출_금액,영역_면적,cluster,geometry
0,마포구,1,한식음식점,5724917.75,16670,1487,163,0.0,0.0,8.969312e+09,384314,0,"POLYGON ((191627.514 450613.571, 191630.569 45..."
1,강동구,2,한식음식점,4547479.75,14552,2169,173,143.0,19.0,1.094268e+10,497570,0,"POLYGON ((211319.234 448645.070, 211324.441 44..."
2,강북구,3,한식음식점,6274558.25,7311,3873,298,160.0,30.0,2.258324e+10,361277,0,"POLYGON ((201732.035 459543.556, 201732.443 45..."
3,구로구,4,한식음식점,1598847.25,5758,537,54,0.0,0.0,1.578294e+09,191612,0,"POLYGON ((186573.498 444690.705, 186559.573 44..."
4,송파구,5,한식음식점,2671650.50,7559,2069,85,45.0,6.0,5.948167e+09,241319,0,"POLYGON ((212860.894 443951.859, 212861.265 44..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,마포구,988,한식음식점,553101.00,431,150,55,0.0,0.0,2.956385e+09,35891,3,"POLYGON ((193023.022 450257.008, 193022.775 45..."
1013,강서구,989,한식음식점,253574.75,348,36,32,0.0,0.0,2.259571e+09,21265,3,"POLYGON ((186226.920 449255.404, 186199.349 44..."
1014,광진구,990,한식음식점,262797.25,900,52,19,0.0,0.0,1.080390e+09,18616,3,"POLYGON ((206036.390 449196.195, 206036.124 44..."
1015,중구,991,한식음식점,700684.00,1234,203,29,0.0,0.0,9.352953e+08,39709,3,"POLYGON ((201820.431 451845.209, 201808.745 45..."


#### 평가 점수 부여

In [25]:
def evaluation_score(df,score1,score2,score3,score4,score5_1,score5_2,score5_3):
    df[['집객력(점수)','매출액(점수)','경쟁업체(점수)','편의성(점수)','잠재고객(점수)']] = 0
    # 집객력
    for i in range(len(df)):
        if df['집객시설수'][i].sum()<=score1[0]:
            df['집객력(점수)'][i] = 1
        elif df['집객시설수'][i].sum()<=score1[1]:
            df['집객력(점수)'][i] = 2
        elif df['집객시설수'][i].sum()<=score1[2]:
            df['집객력(점수)'][i] = 3
        elif df['집객시설수'][i].sum()<=score1[3]:
            df['집객력(점수)'][i] = 4
        else:
            df['집객력(점수)'][i] = 5
        
        # 매출액
        if df['당월_매출_금액'][i].sum()<=score2[0]:
            df['매출액(점수)'][i] = 1
        elif df['당월_매출_금액'][i].sum()<=score2[1]:
            df['매출액(점수)'][i] = 2
        elif df['당월_매출_금액'][i].sum()<=score2[2]:
            df['매출액(점수)'][i] = 3
        elif df['당월_매출_금액'][i].sum()<=score2[3]:
            df['매출액(점수)'][i] = 4
        else:
            df['매출액(점수)'][i] = 5
    
        # 경쟁업체
        if df['유사_업종_점포_수'][i].sum()<=score3[0]:
            df['경쟁업체(점수)'][i] = 1
        elif df['유사_업종_점포_수'][i].sum()<=score3[1]:
            df['경쟁업체(점수)'][i] = 2
        elif df['유사_업종_점포_수'][i].sum()<=score3[2]:
            df['경쟁업체(점수)'][i] = 3
        elif df['유사_업종_점포_수'][i].sum()<=score3[3]:
            df['경쟁업체(점수)'][i] = 4
        else:
            df['경쟁업체(점수)'][i] = 5
    
        # 편의성
        if df['교통시설수'][i].sum()<=score4[0]:
            df['편의성(점수)'][i] = 1
        elif df['교통시설수'][i].sum()<=score4[1]:
            df['편의성(점수)'][i] = 2
        elif df['교통시설수'][i].sum()<=score4[2]:
            df['편의성(점수)'][i] = 3
        elif df['교통시설수'][i].sum()<=score4[3]:
            df['편의성(점수)'][i] = 4
        else:
            df['편의성(점수)'][i] = 5
    
        # 잠재고객
        ## 상주인구
        if df['총_상주인구_수'][i].sum()<=score5_1[0]:
            df['잠재고객(점수)'][i] += 1
        elif df['총_상주인구_수'][i].sum()<=score5_1[1]:
            df['잠재고객(점수)'][i] += 2
        elif df['총_상주인구_수'][i].sum()<=score5_1[2]:
            df['잠재고객(점수)'][i] += 3
        elif df['총_상주인구_수'][i].sum()<=score5_1[3]:
            df['잠재고객(점수)'][i] += 4
        else:
            df['잠재고객(점수)'][i] += 5
        
        ## 유동인구
        if df['총_유동인구_수'][i].sum()<=score5_2[0]:
            df['잠재고객(점수)'][i] += 1
        elif df['총_유동인구_수'][i].sum()<=score5_2[1]:
            df['잠재고객(점수)'][i] += 2
        elif df['총_유동인구_수'][i].sum()<=score5_2[2]:
            df['잠재고객(점수)'][i] += 3
        elif df['총_유동인구_수'][i].sum()<=score5_2[3]:
            df['잠재고객(점수)'][i] += 4
        else:
            df['잠재고객(점수)'][i] += 5
        
        ## 직장인구
        if df['총_직장_인구_수'][i].sum()<=score5_3[0]:
            df['잠재고객(점수)'][i] += 1
        elif df['총_직장_인구_수'][i].sum()<=score5_3[1]:
            df['잠재고객(점수)'][i] += 2
        elif df['총_직장_인구_수'][i].sum()<=score5_3[2]:
            df['잠재고객(점수)'][i] += 3
        elif df['총_직장_인구_수'][i].sum()<=score5_3[3]:
            df['잠재고객(점수)'][i] += 4
        else:
            df['잠재고객(점수)'][i] += 5

    df['잠재고객(점수)'] = df['잠재고객(점수)'] / 3
    
    return df
        
        

In [26]:
score1 = [3,5,12,23]
score2 = [270000000,630000000,1280000000,2900000000]
score3 = [8,16,26,49]
score4 = [0,2,5,9]
score5_1 = [500,668,1655,3361]
score5_2 = [100000,230000,580000,1000000]
score5_3 = [50,111,321,894]

In [27]:
df_final = evaluation_score(df_all,score1,score2,score3,score4,score5_1,score5_2,score5_3)

In [31]:
df_final['geometry']

0      POLYGON ((191627.514 450613.571, 191630.569 45...
1      POLYGON ((211319.234 448645.070, 211324.441 44...
2      POLYGON ((201732.035 459543.556, 201732.443 45...
3      POLYGON ((186573.498 444690.705, 186559.573 44...
4      POLYGON ((212860.894 443951.859, 212861.265 44...
                             ...                        
988    POLYGON ((193023.022 450257.008, 193022.775 45...
989    POLYGON ((186226.920 449255.404, 186199.349 44...
990    POLYGON ((206036.390 449196.195, 206036.124 44...
991    POLYGON ((201820.431 451845.209, 201808.745 45...
992    POLYGON ((197796.320 450210.371, 197795.364 45...
Name: geometry, Length: 993, dtype: geometry

In [32]:
import pandas as pd
df = pd.read_csv('C:/Users/Admin/Downloads/redefined_df.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9231 entries, 0 to 9230
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   상권_코드        9231 non-null   int64  
 1   서비스_업종_코드_명  9231 non-null   object 
 2   총_유동인구_수     9231 non-null   float64
 3   총_상주인구_수     9231 non-null   int64  
 4   총_직장_인구_수    9231 non-null   int64  
 5   유사_업종_점포_수   9231 non-null   int64  
 6   집객시설수        9231 non-null   float64
 7   교통시설수        9231 non-null   float64
 8   당월_매출_금액     9231 non-null   float64
 9   영역_면적        9231 non-null   int64  
 10  cluster      9231 non-null   int64  
 11  geometry     9231 non-null   object 
 12  집객력(점수)      9231 non-null   int64  
 13  매출액(점수)      9231 non-null   int64  
 14  경쟁업체(점수)     9231 non-null   int64  
 15  편의성(점수)      9231 non-null   int64  
 16  잠재고객(점수)     9231 non-null   float64
dtypes: float64(5), int64(10), object(2)
memory usage: 1.2+ MB


In [38]:
df = gpd.GeoDataFrame(df)
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9231 entries, 0 to 9230
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   상권_코드        9231 non-null   int64  
 1   서비스_업종_코드_명  9231 non-null   object 
 2   총_유동인구_수     9231 non-null   float64
 3   총_상주인구_수     9231 non-null   int64  
 4   총_직장_인구_수    9231 non-null   int64  
 5   유사_업종_점포_수   9231 non-null   int64  
 6   집객시설수        9231 non-null   float64
 7   교통시설수        9231 non-null   float64
 8   당월_매출_금액     9231 non-null   float64
 9   영역_면적        9231 non-null   int64  
 10  cluster      9231 non-null   int64  
 11  geometry     9231 non-null   object 
 12  집객력(점수)      9231 non-null   int64  
 13  매출액(점수)      9231 non-null   int64  
 14  경쟁업체(점수)     9231 non-null   int64  
 15  편의성(점수)      9231 non-null   int64  
 16  잠재고객(점수)     9231 non-null   float64
dtypes: float64(5), int64(10), object(2)
memory usage: 1.2+ MB
